In [1]:
import pandas as pd 
import numpy as np

import os

In [4]:
os.listdir('../outputs/mistral-7B/few_shot_cot_v1/results.csv')

NotADirectoryError: [Errno 20] Not a directory: '../outputs/mistral-7B/few_shot_cot_v1/results.csv'

In [18]:
results = pd.read_csv('../outputs/mistral-7B/one_shot_cot_v1_longer_output/results.csv')

In [32]:
(results['generated_text'] == 'The answer is X.').sum()

np.int64(45)

In [28]:
results.iloc[97].generated_text

'The answer is X.'

In [27]:
results.iloc[97].model_prediction

np.float64(nan)